In [4]:
#!pip install imagehash Pillow 
#!pip install imagehash 


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [27]:
import os
import glob
from PIL import Image
import imagehash
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
import pandas as pd 

In [23]:


# Install the required libraries
# !pip install imagehash Pillow scikit-learn

def read_images_from_folder(folder_path):
    image_paths = glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png'))
    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        images.append(image)
    return images, image_paths

def compute_image_hashes(images):
    image_hashes = []
    for image in images:
        image_hash = imagehash.phash(image)
        image_hashes.append(image_hash)
    return image_hashes

def cluster_images(image_hashes, n_clusters, linkage='ward'):
    hash_arrays = [np.array(hash.hash.flatten(), dtype=bool) for hash in image_hashes]
    distance_matrix = squareform(pdist(np.stack(hash_arrays), metric='hamming'))
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage=linkage)
    labels = clustering.fit_predict(distance_matrix)
    return labels

def cluster_images(image_hashes, n_clusters, linkage='average'):
    hash_arrays = [np.array(hash.hash.flatten(), dtype=bool) for hash in image_hashes]
    distance_matrix = squareform(pdist(np.stack(hash_arrays), metric='hamming'))
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage=linkage)
    labels = clustering.fit_predict(distance_matrix)
    return labels


def group_images_by_cluster(labels, image_paths):
    clusters = {}
    for label, image_path in zip(labels, image_paths):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(image_path)
    return clusters

def main(folder_path, n_clusters):
    images, image_paths = read_images_from_folder(folder_path)
    image_hashes = compute_image_hashes(images)
    labels = cluster_images(image_hashes, n_clusters)
    clusters = group_images_by_cluster(labels, image_paths)
    result_df = pd.DataFrame({'image_path': image_paths, 'cluster_label': labels})
    return result_df





In [25]:
pd.set_option('display.max_colwidth', 100)

In [26]:
folder_path = '/workspaces/predictive_maintenance/data/images/'
n_clusters = 2
main(folder_path, n_clusters)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


,image_path,cluster_label
0,/workspaces/predictive_maintenance/data/images/dog.111.jpg,0
1,/workspaces/predictive_maintenance/data/images/dog.106.jpg,1
2,/workspaces/predictive_maintenance/data/images/0002.png,0


/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


,image_path,cluster_label
0,/workspaces/predictive_maintenance/data/images...,1
1,/workspaces/predictive_maintenance/data/images...,0
